## 9.2 装饰器基本知识

1. 装饰器是一个函数或其他可调用对象；
2. 装饰器可以把被装饰的函数替换为另一个函数；
3. 装饰器在加载模块时立即执行

In [4]:
def deco(func):
    def inner():
        print("this is the inner function")
        return "inner function"
    return inner
@deco
def target():
    print("this is the origin funcction")
    return "target"

target() # This will print "this is the inner function" and return "inner function"


this is the inner function


'inner function'

## 9.6 闭包

In [12]:
def deco_wrapper():
    temp_list = []
    def wrapper(*args, **kwargs):
        temp_list.append(1)
        print(temp_list)
        return temp_list
    return wrapper
a = deco_wrapper()
a()
a()
a()


[1]
[1, 1]
[1, 1, 1]


[1, 1, 1]

In [ ]:
def deco_2():
    temp_str = 0
    def wrapper():
        nonlocal temp_str # 将temp_str声明为非局部变量
        temp_str += 1
        print(temp_str)
        return temp_str
    return wrapper
b = deco_2()
b()

## 9.8 实现简单的装饰器

In [19]:
import time
def deco_time(func):
    count = 0
    def wrapper(*args, **kwargs):
        nonlocal count
        start = time.time()
        print("Starting function execution...")
        result = func(*args, **kwargs)
        end = time.time()
        count += 1
        print(f"Function {func.__name__} called {count} times, took {end - start:.4f} seconds")
        return result
    return wrapper
@deco_time
def example_function(x, y):
    print("Executing example_function...")
    time.sleep(1)

In [22]:
example_function(1, 2)

Starting function execution...
Executing example_function...
Function example_function called 3 times, took 1.0034 seconds


#### functools wraps

In [23]:
# 改进后的装饰器
import functools
def deco_time_improved(func):
    @functools.wraps(func)
    def wrapper(*args,**kwargs):
        print("Before function start")
        result = func(*args,**kwargs)
        print("After function done")
    return wrapper

@deco_time_improved
def test(num:int):
    print(f'程序输入：{num}')
    
test(2)

Before function start
程序输入：2
After function done


## 9.9 标准库中的装饰器

### 9.9.1 使用 functools.cache 来做备忘

- 如果缓存较大，则functools.cache有可能耗尽所有可用内存。在我看来，@cache更适合短期运行的命令行脚本使用。对于长期运行的进程，推荐使用functools.lru_cache，并合理设置maxsize参数

In [1]:
import time
import functools
def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked

@functools.cache
@clock
def cal(num:int):
    time.sleep(4)
    return num

In [2]:
cal(4)


[4.00505054s] cal(4) -> 4


4

In [3]:
cal(4)

4

In [4]:
cal(3)

[4.00510150s] cal(3) -> 3


3

### 9.9.2 使用 functools.lru_cache

### 9.9.3 单分派泛化函数

- 应尽量注册处理抽象基类（例如numbers.Integral和abc.MutableSequence）的专门函数，而不直接处理具体实现（例如int和list）。这样的话，代码支持的兼容类型更广泛。例如，Python扩展可以子类化numbers.Integral，使用固定的位数实现int类型。

In [13]:
from functools import singledispatch
import numbers
from collections import abc
from numpy import number

@singledispatch # 这个装饰器用于注册不同类型的处理函数
def print_type(arg):
    # 默认处理函数，如果没有注册特定类型的处理函数，则调用此函数
    print(f"Unsupported type: {type(arg)}")

@print_type.register # 使用register方法注册不同类型的处理函数
def _(arg: numbers.Integral):
    print(f"Integer: {arg}")

@print_type.register
def _(arg: bool):
    print(f"Boolean: {arg}")

@print_type.register
def _(arg: str):
    print(f"String: {arg}")

@print_type.register
def _(arg: abc.Sequence):
    print(f"List: {arg}")

@print_type.register(float)
def _(arg):
    print(f"Float: {arg}")    

In [14]:
print_type(42)        # Integer: 42
print_type(True)      # Boolean: True
print_type({"aaa":42}) # Unsupported type: <class 'dict'>
print_type([1,2,3])
print_type("Hello")   # String: Hello
print_type(3.14)      # Float: 3.14
print_type(3.14j)     # Unsupported type: <class 'complex'>

Integer: 42
Boolean: True
Unsupported type: <class 'dict'>
List: [1, 2, 3]
String: Hello
Float: 3.14
Unsupported type: <class 'complex'>


## 9.10 参数化装饰器

In [ ]:
def deco_3(flag:str):
    def real_deco(func):
        def wrapper(*args, **kwargs):
            print(f"装饰器参数：{flag}")
            result = func(*args, **kwargs)
            print(f"装饰器返回值：{result}")
            return result
        return wrapper
    return real_deco
@deco_3("test")
def target_2(arg):
    print(f"this is the target_2 function")
    return f"{arg}"
target_2("hello")  # This will print "this is the target_2 function" and return "hello"

装饰器参数：test
this is the target_2 function
装饰器返回值：hello


'hello'

### 9.10.2 参数化 clock 装饰器

In [ ]:
def clock(fmt = "{name}({args}) -> {result}"):
    def decorator(func):
        def clocked(**kwargs):
            name = func.__name__
            args = [repr(arg) for arg in kwargs]
            result = func(**kwargs)
            print(fmt.format(**locals()))
            return result
        return clocked
    return decorator
@clock(fmt="{name}({args}) -> {result}")
def test(num=1):
    return num + 1
test(num=3)

test(["'num'"]) -> 4


4

### 9.10.3 基于类的 clock 装饰器

In [ ]:
# 改写 9.10.2 的装饰器，基于类实现
class Clock():
    def __init__(self,fmt):
        self.fmt = fmt
    def __call__(self,func):
        def clocked(**kwargs):
            name = func.__name__
            args = [repr(arg) for arg in kwargs]
            result = func(**kwargs)
            print(self.fmt.format(**locals()))
            return result
        return clocked
@Clock(fmt="{name}({args}) -> {result}")
def test(num=1):
    return num + 1
test(num=3)

test(["'num'"]) -> 4


4